# 1. Main script
## It contains:
- Calculations only for one subpath, because of time complexity
- The data 
- Calls to each method
- Final results

In [ ]:
import pandas as pd
import numpy as np
import math

# 2. Read data and make them easier to understand

## 2.1. GSE2034

In [ ]:
raw_gse2034_df = pd.read_csv('Data/GSE2034.zip', compression='zip', header=0, sep='\t', quotechar='"') # Breast cancer
gse2034_df=raw_gse2034_df.copy()

# preprocess dataset
gse2034_df[['Gene','KEGG-ID']] = gse2034_df['Class'].str.split('#',expand=True)
gse2034_df.drop('Class', inplace=True, axis=1)
cols = gse2034_df.columns.tolist()
cols = cols[-2:] + cols[:-2]
gse2034_df=gse2034_df[cols]

#gse2034_df

In [ ]:
# Change column names (estrogen receptor)
labels=gse2034_df.columns[2:]
for x in range(len(labels)):
    if(labels[x].startswith('ERpos')):
        labels.values[x]="ERpos"
    elif(labels[x].startswith('ERneg')):
        labels.values[x]="ERneg"

In [ ]:
# Get the genes that are present in the GSE2034 dataset and create a dictionary 
# where the keys are the genes and the values are the corresponding KEGG-IDs
# 'a gene can be mapped to more than one Entrez identifier'
gene_list=sorted(set(gse2034_df['Gene'].tolist()))
gene_dict={}
for i in gene_list:
    tmp=(gse2034_df.loc[gse2034_df['Gene'] == i]['KEGG-ID']).copy()
    tmp_list=[]
    for t in tmp:
        tmp_list.append(t)
    gene_dict.update({i:tmp_list})
#print('Gene dictionary (key: Genes, values: KEGG-IDs): '+str(gene_dict))

In [ ]:
# transpose dataframe so that the columns indicate the genes
# and rows correspond to samples (class: ERpos or ERneg)
genes=(gse2034_df['Gene']).copy()
gse2034_df.drop('KEGG-ID', inplace=True, axis=1)
gse2034_df=np.transpose(gse2034_df.iloc[:,1:])
gse2034_df.columns=genes.values.tolist()
#gse2034_df

In [ ]:
# Because one gene might correspond to more than one KEGG-IDs, we calculate the average (or max)
# value and get the following simplified dataframe
gse2034_df=gse2034_df.groupby(level=0,axis=1).mean()
gse2034_df['noProbe']=gse2034_df.mean(axis=1) # Compute 'noProbe' for future use
gse2034_df

## 2.1.1. Split data in training and testing sets (size reduction)
- Training set size: 70%
- Testing set size: remaining 30%

In [ ]:
from sklearn.model_selection import train_test_split

training_data, testing_data, training_labels, testing_labels = train_test_split(gse2034_df, gse2034_df.index, test_size = 0.3 ,random_state = 0)

## 2.2. Selected
Cellular processes (15), Signal transduction (Environmental information process) (24), Cancer overview (8).

In [ ]:
raw_selected_df = pd.read_csv('Data/Selected.zip', compression='zip', header=0, sep='\t', quotechar='"')[['SubPathID']]
#raw_selected_df

In [ ]:
# Two types of nodes relations
relations_dict={'Activation':'-->','Inhibition':'--|'}
#relations_dict

In [ ]:
from itertools import chain

# Split each pathway based on the relation
def split_path(data,relation):
    s=[]
    cnt=len(data.split(relations_dict[relation]))
    cnt_tmp=1
    # If there is at least one relation, then split data
    if(cnt>0):
        for e in data.split(relations_dict[relation]):
            if e:
                s.append(e)
                # Remove the final relation
                if(cnt_tmp<cnt):
                    s.append(relations_dict[relation])
                cnt_tmp+=1
    return s

# Make the selected dataset easier to understand, by splitting each pathway based on their nodes and edges
def get_pathway(data):
    s=split_path(data,'Activation')

    for i in range(len(s)):
        tmp_s=split_path(s[i],'Inhibition')
        if(len(s[i])>1):
            s[i]=tmp_s
            
    return list(chain.from_iterable(s))

subpaths_list=[get_pathway(row) for row in raw_selected_df['SubPathID']]
#subpaths_list

In [ ]:
selected_df=pd.DataFrame(subpaths_list).fillna(value=np.nan) # Rows: pathways, Cols: edges and nodes
selected_df

## 2.3. Important values

### 2.3.1. Node genes- all genes of each node

In [ ]:
# Each node in a pathway represents a discrete function mapping to one or more transcript.
# Returns a dictionary corresponding each node of each pathway to its gene or genes.
def Node_genes(df):
    Node_genes={}
    for path in range(df.shape[0]):
        tmp_node=[]
        for node in range(0,len(df.iloc[path][~df.iloc[path].isnull()]),2):
            genes=list(filter(None,[x.strip() for x in df.iloc[path,node].split(' ')]))
            tmp_genes=[]
            for g in genes:
                tmp_genes.append(list(filter(None,[x.strip() for x in g.split('#')]))[0])
            tmp_node.append(tmp_genes)
        Node_genes.update({path:tmp_node})
    return Node_genes

node_genes=Node_genes(selected_df)
node_genes

### 2.3.2. Expression value
For each subpath assign each sample's genes expression values (3D: samples,subpaths,nodes)

In [ ]:
from statistics import mean

# For a specific subpath match the expression values of each gene based on the values in a specific sample.
# In case of multiple genes in one node assign the average value.
def sample_expression_value(subpath,sample):
    sample_expr_val=[]
    for node in range(len(subpath)):
        node_genes=[]
        for gene in range(len(subpath[node])):
            node_genes.append(sample[gene])
        sample_expr_val.append(mean(node_genes)) # Calculate the average expression value of a node's genes
    return sample_expr_val

#sample_no=0
#path_no=0
#sample_expression_value(node_genes[path_no],training_data.iloc[sample_no])

In [ ]:
# For each subpath assign the expression values based on each sample and return a 2D array
def subpath_expression_value(subpath,samples):
    
    # Create a 2D array, where rows are the sample size and columns are the subpath's nodes
    subpath_expr_val=[]
    
    # For each sample
    for sample in range(samples.shape[0]):
        subpath_expr_val.append(list(samples.iloc[sample]))
        
    return subpath_expr_val

#path_no=1
#subpath_expression_value(node_genes[path_no],training_data)

In [ ]:
def get_expression_value(node_genes,samples):
    expr_val={}
    for path in node_genes:
        print(path)
        expr_val.update({path:subpath_expression_value(node_genes[path],samples)})
    return expr_val

#expression_values=get_expression_value(node_genes,training_data)
#expression_values

Because of the time complexity, the expression value is calculated only for one subpath

In [ ]:
path_no=0
subpath_expr_val=subpath_expression_value(node_genes[path_no],training_data)
subpath_expr_val

### 2.3.3. P-value and threshold <= 0.05 (gene is significant)
For each gene find the p-value (ttest_ind)

In [ ]:
import scipy.stats as stats

gse2034_df2=gse2034_df.copy().T # Columns: samples and rows: genes
gse2034_df2

#### 2.3.3.1. For each gene find cancer and normal mean value and calculate their difference

In [ ]:
gse2034_df2['ERnegMean']=gse2034_df2['ERneg'].mean(axis=1)
gse2034_df2['ERposMean']=gse2034_df2['ERpos'].mean(axis=1)
gse2034_df2['Diffs']=gse2034_df2['ERnegMean']-gse2034_df2['ERposMean']
gse2034_df2

In [ ]:
def pvalue005(genes):
    pvalArray=np.empty(genes.shape[0])
    pvalArrayUnder005=np.array([])
    indices005=np.array([])
    for x in range(genes.shape[0]):
        st,pval=stats.ttest_ind(genes['ERneg'].iloc[x],genes['ERpos'].iloc[x])
        pvalArray[x]=pval
        if(pval<0.05):
            pvalArrayUnder005=np.append(pvalArrayUnder005,pval)
            indices005=np.append(indices005,x)
    return pvalArray,pvalArrayUnder005,indices005

pvalList2,pvalListUnder005_2,indices005_2=pvalue005(gse2034_df2)

In [ ]:
gse2034_df2['P-Value']=pvalList2
gse2034_df2

In [ ]:
# For each node of a sub-path consisting of more than one genes, get the average value of the p-values.
def path_pvalue(path,df):
    cur_path=[]
    for node in path:
        cur_node=[]
        for gene in node:
            # Check if gene is not in genes' list
            if(not(gene in df.index)):
                cur_node.append(df.loc['noProbe']['P-Value'])
                continue
            cur_node.append(df.loc[gene]['P-Value'])
        cur_path.append(np.mean(cur_node))
    return cur_path
         
def get_pvalues(node_genes,df):
    pvalues={}
    for path in node_genes:
        pvalues.update({path:path_pvalue(node_genes[path],df)})  
    return pvalues

pvalue_threshold=0.05

#pvalues_dict=get_pvalues(node_genes,gse2034_df2)
#pvalues_dict

pvalues=path_pvalue(node_genes[path_no],gse2034_df2)
pvalues

### 2.3.4. Fold Change and Log Fold Change
Add fold change column to genes_df

In [ ]:
# Calculate fold change for each gene
def fold_change(df):
    fc_dict={}
    
    for gene in df.index:
        # Get average value for each case
        erneg_av=df.loc[gene]['ERneg'].mean()
        erpos_av=df.loc[gene]['ERpos'].mean()
        
        # Calculate fold change (B/A)
        cur_fc=erneg_av/erpos_av
        fc_dict.update({gene:cur_fc})
        
    #fc_dict.update({'noProbe':mean(list(fc_dict.values()))})
        
    return fc_dict        

fc=fold_change(gse2034_df2)
gse2034_df2['Fold Change']=fold_change(gse2034_df2).values()
gse2034_df2['Log FC']=[math.log(fc+1-min(gse2034_df2['Fold Change'])) for fc in gse2034_df2['Fold Change']] # Calculate log fold change
gse2034_df2

In [ ]:
# For each node of a sub-path consisting of more than one genes, get the average value of the fold change.
def path_fc(path,df):
    fc=[]
    log_fc=[]
    for node in path:
        cur_fc=[]
        cur_log_fc=[]
        for gene in node:
            # Check if gene is not in genes' list
            if(not(gene in df.index)):
                cur_fc.append(df.loc['noProbe']['Fold Change'])
                cur_log_fc.append(df.loc['noProbe']['Log FC'])
                continue
            cur_fc.append(df.loc[gene]['Fold Change'])
            cur_log_fc.append(df.loc[gene]['Log FC'])
        fc.append(np.mean(cur_fc))
        log_fc.append(np.mean(cur_log_fc))
    return fc,log_fc
            

def get_fc(node_genes,df):
    fc={}
    log_fc={}
    for path in node_genes:
        cur_fc,cur_log_fc=path_fc(node_genes[path],df)
        fc.update({path:cur_fc})  
        log_fc.update({path:cur_log_fc})  
    return fc,log_fc

log_fc_threshold=1.5 

#fc_dict,log_fc_dict=get_fc(node_genes,gse2034_df2) # Return fold change and log fold change in dictionary form

fc,log_fc=path_fc(node_genes[path_no],gse2034_df2)

### 2.3.5. Differentially Expressed Genes (DEG)

In [ ]:
# Get the differentialy expressed genes as dictionary
def path_de_genes(pvalues):
    de_genes=[]
    for node in range(len(pvalues)):
        if(pvalues[node]<=pvalue_threshold):
            de_genes.append(1)
        else:
            de_genes.append(0)
    return de_genes

def get_de_genes(pvalues_dict):
    de_genes={}
    for path in pvalues_dict:
        de_genes.update({path:path_de_genes(pvalues_dict[path])})
    return de_genes

#de_genes_dict=get_de_genes(pvalues_dict)

de_genes=path_de_genes(pvalues)
de_genes

# 3. Methods

## 3.1. PRS

In [ ]:
from ipynb.fs.full.PRS import *

In [ ]:
path_PRS(log_fc,fc,pvalues)

## 3.2. MinePath

In [ ]:
from ipynb.fs.full.MinePath import *

In [ ]:
sample_midpoints(gse2034_df.iloc[0])

In [ ]:
get_midpoints(gse2034_df)